In [9]:
%matplotlib inline
from __future__ import print_function, division
import matplotlib.pylab as plt
import numpy as np
from keras.optimizers import Adam
import os
import glob
import io
from PIL import Image
import sys

from scipy.misc import imsave

from tensorflow.spectral import dct, idct
from scipy.fftpack import dct as DCT, idct as IDCT
from tensorflow.keras.backend import eval
import skimage.measure as metrics

# Importing our U-Net model
MY_UTILS_PATH = "../../Modules/"
if not MY_UTILS_PATH in sys.path:
    sys.path.append(MY_UTILS_PATH)
from models import unet_old, unet, my_unet

In [10]:
def dct2d(im):
    return DCT(DCT(im,type=2,axis=0,norm='ortho'),type=2,axis=1,norm='ortho')
def idct2d(mat):
    return IDCT(IDCT(mat,type=2,axis=0,norm='ortho'),type=2,axis=1,norm='ortho')

def dct_in_blocks(im, block_size = 8):

    rows, cols = im.shape[0], im.shape[1]

    # block size: 8x8
    if rows % block_size == cols % block_size == 0:
        blocks_count = rows // block_size * cols // block_size
    else:
        raise ValueError(("the width and height of the image "
                          "should both be mutiples of %block_size"))

    dct_matrix = np.zeros((rows, cols))

    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):

            block = im[i:i+block_size, j:j+block_size]
            dct_matrix[i:i+block_size,j:j+block_size] = dct2d(block)

    return dct_matrix

def idct_in_blocks(dct_mat, block_size = 8):

    rows, cols = dct_mat.shape[0], dct_mat.shape[1]

    # block size: 8x8
    if rows % block_size == cols % block_size == 0:
        blocks_count = rows // block_size * cols // block_size
    else:
        raise ValueError(("the width and height of the image "
                          "should both be mutiples of %block_size"))

    im_matrix = np.zeros((rows, cols))

    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):

            block = dct_mat[i:i+block_size, j:j+block_size]
            im_matrix[i:i+block_size,j:j+block_size] = idct2d(block)

    return im_matrix


In [32]:
# Train and validation data paths
train_path = "/home/ubuntu/volume1/TIFF/Development/Training/*.tif"
val_path = "/home/ubuntu/volume1/TIFF/Development/Validation/*.tif"
test_path = "/home/ubuntu/volume1/TIFF/Testing/*.tif"


# Listing train and validation images
train_images = sorted(glob.glob(train_path))
val_images = sorted(glob.glob(val_path))
test_images = sorted(glob.glob(test_path))

# Summary of images
print("Train set size:", len(train_images))
print("Validation set size:",len(val_images))
print("Test set size:",len(test_images))

# images_path = "../../../Gray/*.tif"
# images = glob.glob(images_path)
# print(len(images))
# # Listing train and validation images
# train_images = images[0:30000]
# val_images = images[30000:45000]
# test_images = images[45000:]

# # Summary of images
# print("Train set size:", len(train_images))
# print("Validation set size:",len(val_images))
# print("Test set size:",len(test_images))


Train set size: 10740
Validation set size: 5370
Test set size: 16110


In [33]:
test_images = train_images

In [34]:
H,W = 256, 256
norm = 255.0
compr_range = (10,20)

test_unc = np.zeros((len(test_images),H,W,1), dtype=np.float32)
test_compr = test_unc.copy()

dct_test_unc = test_unc.copy()
dct_test_compr = test_unc.copy()

compr_test = np.random.choice(np.arange(compr_range[0],compr_range[1], dtype=int), \
                               len(test_images),replace = True)

# Test set
for ii in range(len(test_images)):
    # Load uncompressed image
    im1 = Image.open(test_images[ii])
    test_unc[ii,:,:,0] = np.array(im1, dtype=np.float32)

    # JPEG compress the image    
    buf = io.BytesIO()
    im1.save(buf, "JPEG", quality=int(compr_test[ii]))
    im2 = Image.open(buf)
    test_compr[ii,:,:,0] = np.array(im2, dtype=np.float32)

test_unc -= 128.0
test_compr -= 128.0

test_unc /= 128.0
test_compr /= 128.0

In [35]:
useBlocks = True

if useBlocks:
    for ii in range(len(test_images)):
        dct_test_compr[ii,:,:,0] = dct_in_blocks(test_compr[ii,:,:,0])
else:
    for ii in range(len(test_images)):
        dct_test_compr[ii,:,:,0] = dct2d(test_compr[ii,:,:,0])
    
# # keep min/max values to de-normalize later
# dct_test_compr_min = np.zeros(len(test_images))
# dct_test_compr_max = np.zeros(len(test_images))

# for ii in range(len(test_images)):
#     dct_test_compr_min[ii] = np.min(dct_test_compr[ii,:,:,0])
#     dct_test_compr_max[ii] = np.max(dct_test_compr[ii,:,:,0])

# print(dct_test_compr_min.shape)
# print(dct_test_compr_max.shape)

In [36]:
opt=Adam(lr=0.0001)
model_name = "../../Models/residual_unet_DCT_data_augmentation_compress_10_20_2019-03-25-pretrained.hdf5"
#model = unet_old(input_size = (256 ,256 ,1),drop = 0.0,residual = True)
model=my_unet()
model.compile(loss = "mse",optimizer=opt)

model.load_weights(model_name)

In [37]:
pred = model.predict(dct_test_compr)

In [38]:
img_pred = test_unc.copy()

test_unc *= 128.0
test_unc += 128.0

test_compr *= 128.0
test_compr += 128.0

In [39]:
if useBlocks:
    for ii in range(len(test_images)):
        img_pred[ii,:,:,0] = idct_in_blocks(pred[ii,:,:,0])
else:
    for ii in range(len(test_images)):
        img_pred[ii,:,:,0] = idct2d(pred[ii,:,:,0])
        
img_pred *= 128.0
img_pred += 128.0

In [40]:
# Save images as numpy arrays
np.save("../../np_imagedata/train_residual_pseudo_wnet_i_data_augmentation_compress_10_20_2019-03-25-pretrained.npy",img_pred)

In [ ]:
out_path = '/home/ubuntu/volume1/TIFF/Testing_dct_unet'
for ii in range(len(test_images)):
    name = test_images[ii].split("/")[-1]
    path = os.path.join(out_path,name)
    imsave(path,img_pred[ii,:,:,0])
    
# out_path = '/home/ubuntu/volume1/TIFF/Development_dct_unet/Validation'
# for ii in range(len(test_images)):
#     name = test_images[ii].split("/")[-1]
#     path = os.path.join(out_path,name)
#     imsave(path,img_pred[ii,:,:,0])
    
# out_path = '/home/ubuntu/volume1/Gray_unet'
# for ii in range(len(test_images)):
#     name = test_images[ii].split("/")[-1]
#     path = os.path.join(out_path,name)
#     imsave(path,img_pred[ii,:,:,0])
    
#     im = Image.fromarray(img_pred[ii,:,:,0])
#     im.save(path)

In [ ]:
see_image = glob.glob('/home/ubuntu/volume1/TIFF/Testing_dct_unet/*.tif')
num = 40
testimg = Image.open(see_image[num])
uncimg = Image.open(test_images[num])

buf = io.BytesIO()
uncimg.save(buf, "JPEG", quality=int(10))
jpegimg = Image.open(buf)
    
plt.figure(figsize=(18,9))
plt.subplot(1,3,1)
plt.imshow(np.array(uncimg), cmap="gray")
plt.axis("off")
plt.title("Uncompressed")
plt.subplot(1,3,2)
plt.imshow(np.array(jpegimg), cmap="gray")
plt.axis("off")
plt.title("JPEG Decompression")
plt.subplot(1,3,3)
plt.imshow(np.array(testimg), cmap="gray")
plt.axis("off")
plt.title("U-Net Decompression")
plt.show()